<a href="https://colab.research.google.com/github/elpiprin/NLP-Stuff/blob/main/RNN_Custom1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

***Imports***

In [3]:
#https://www.tensorflow.org/text/tutorials/text_generation
import tensorflow as tf

import numpy as np
import os
import time

***Reading the data***

In [5]:
#Read, then decode for py2 compat.
text = open('/content/Articles.txt', 'rb').read().decode(encoding='utf-8')
#length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 480143 characters


In [6]:
#Take a look at the first 250 characters in text
print(text[:250])

Melissa Caddick: Death of Australian fraudster remains a mystery

When conwoman Melissa Caddick vanished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed foot found washed up on a beach months later - it 


In [7]:
#The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

105 unique characters


### **Processing the text**

***Vectorising the text***

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

***Prediction task*** 

***Creating training examples and targets*** 

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(480143,), dtype=int64, numpy=array([43, 63, 70, ..., 76, 77, 16])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

M
e
l
i
s
s
a
 
C
a


In [18]:
seq_length = 100

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'M' b'e' b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c'
 b'k' b':' b' ' b'D' b'e' b'a' b't' b'h' b' ' b'o' b'f' b' ' b'A' b'u'
 b's' b't' b'r' b'a' b'l' b'i' b'a' b'n' b' ' b'f' b'r' b'a' b'u' b'd'
 b's' b't' b'e' b'r' b' ' b'r' b'e' b'm' b'a' b'i' b'n' b's' b' ' b'a'
 b' ' b'm' b'y' b's' b't' b'e' b'r' b'y' b'\r' b'\n' b'\r' b'\n' b'W' b'h'
 b'e' b'n' b' ' b'c' b'o' b'n' b'w' b'o' b'm' b'a' b'n' b' ' b'M' b'e'
 b'l' b'i' b's' b's' b'a' b' ' b'C' b'a' b'd' b'd' b'i' b'c' b'k' b' '
 b'v' b'a' b'n'], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'
b'ished from her luxurious eastern Sydney home in November 2020 - with only her partially decomposed fo'
b'ot found washed up on a beach months later - it set off a frenzy in Australia.\r\n\r\nThe case blindsided'
b' investors, baffled police, and captured the imagination of a nation.\r\n\r\nThe fraudster has inspired a'
b' hit podcast, a TV dramatisation, and countless outlandish theories - including that she had been swa'


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Melissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick va'
Target: b'elissa Caddick: Death of Australian fraudster remains a mystery\r\n\r\nWhen conwoman Melissa Caddick van'


***Creating training batches***

In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Building the model**

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### **Trying the model**

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 106) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  27136     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  108650    
                                                                 
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [32]:
sampled_indices

array([ 25, 104,  78,  26,   2,   7,  73,  45,  65,  21,  64,   2,   4,
       102,   9,  90,   2,  70,  95,  74,  56,  45,  38,  73, 102, 103,
        89, 104,  11,  63,  64,  63,  39,  54,  88,  74,  50,  50,  12,
        19, 104,  95,  39,  91,  37,  86,  88,  33,  61,  37,  79,  16,
        78,  36,   1,  99,  29,  44, 101,  47,  19,  32,  13,  23,  25,
        90,  32,  20,  48,  57,  81, 104,  41,  49,  71,  71,  60,  83,
        39,   2,  46,  31,  15,  60,  70, 102,  63,  81,  82,  15,  85,
        84,  68, 103,  99,  19,  21,  77,  44,  28])

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ent cow slaughter, which is illegal in many Indian states.\r\n\r\nThe police, however, say that there we'

Next Char Predictions:
 b'7\xe2\x80\xaft8\r$oOg3f\r!\xe2\x80\x9d&\xc3\xad\rl\xc3\xbcpZOHo\xe2\x80\x9d\xe2\x80\xa6\xc3\xa9\xe2\x80\xaf(efeIX\xc3\xa8pTT)1\xe2\x80\xaf\xc3\xbcI\xc3\xb1G\xc3\xa1\xc3\xa8CcGu.tF\n\xe2\x80\x98;N\xe2\x80\x9cQ1B*57\xc3\xadB2R[w\xe2\x80\xafKSmmbyI\rPA-bl\xe2\x80\x9dewx-\xc2\xa3zj\xe2\x80\xa6\xe2\x80\x9813sN:'


### **Training the model**

***Attaching an optimiser & a loss function***

In [137]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [138]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 106)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.663675, shape=(), dtype=float32)


In [139]:
tf.exp(example_batch_mean_loss).numpy()

106.024994

In [141]:
model.compile(optimizer='adam', loss=loss)

***Configuring checkpoints***

In [142]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

***Executing the training***

In [145]:
EPOCHS = 60

In [146]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
74/74 [==============================] - 258s 3s/step - loss: 2.3911
Epoch 2/60
74/74 [==============================] - 267s 4s/step - loss: 2.2134
Epoch 3/60
74/74 [==============================] - 256s 3s/step - loss: 2.0554
Epoch 4/60
74/74 [==============================] - 258s 3s/step - loss: 1.9103
Epoch 5/60
74/74 [==============================] - 256s 3s/step - loss: 1.7691
Epoch 6/60
74/74 [==============================] - 255s 3s/step - loss: 1.6419
Epoch 7/60
74/74 [==============================] - 256s 3s/step - loss: 1.5358
Epoch 8/60
74/74 [==============================] - 256s 3s/step - loss: 1.4497
Epoch 9/60
74/74 [==============================] - 255s 3s/step - loss: 1.3772
Epoch 10/60
74/74 [==============================] - 255s 3s/step - loss: 1.3135
Epoch 11/60
74/74 [==============================] - 256s 3s/step - loss: 1.2551
Epoch 12/60
74/74 [==============================] - 257s 3s/step - loss: 1.2007
Epoch 13/60
74/74 [==================

### **Generating Text**

In [34]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [35]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [151]:
#Text 1
start = time.time()
states = None
next_char = tf.constant(['Activists'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Activists and blocking traffic. The sentence sparked engines for exploitation.

Several asylum seekers interviewed by the Guardian say they were forced to return home alive."

The call-up of paratroopers, part of Russia's wider mobilisation, shortages of equipment and medical expertise, which means not everyone gets what they want. That’s the responsibility of governing,” Grantham said.

Ghosts of crises past

Grantham sees uncomfortable parallels between markets to the 737 Max programme do, however, acknowledge that there have been specific issues with stabiliser this event suggest any form of terror threat.”

But the influx of people to a major city gathering in developing new kinds of GPUs moral demand in Africa for vehicles is actually one of the only improvements that I can recall human risk. “The protests agreemed to protest is safe houses in real time," says Njoki. “I stay because it’s very difficult to move elsewhere when you’re employed of the trapping was on the fast during R

In [171]:
#Text 2
start = time.time()
states = None
next_char = tf.constant(['President'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Presidential adviser to the airport. She was not resolved from other sources that the regiment surprises to provide security to Afghans. New data shows threat from ISIS is growing

Qasiman signs suit in basic and banned. I wrong to take a hot long wave of Xi Jinping second by the privacy expert who has involved climate change affects to access mental health services.

Other NHS trusts sent detailed receipts to Facebook. The Information Commissioner’s Office (ICO) is investigating.

The abuse got so bad that in April, AEMET posted a video on Twitter calling on The Swnessor and the US government, an expert in the spotlight, putting its best foot forward and talk to each other before the final legislative text is posted Sunday.

Nob do women are a vote of no confidence.

"Khan results are potentially complacee (32.80 per 100, aged 15, he flight then no longer a female in the region and an invade.

The country — delieve that the mobile networks affected has a company is likely to include s

In [202]:
#Text 3
start = time.time()
states = None
next_char = tf.constant(['Conflict'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Conflict responder by the NHS websites appeared similar to what appears the coroner questions about the health system [on Thursday.

"If someone has a cardiovascular illness, they might otherwise have ended his days.

Now that they earlier of the 131 have thrown holds 85 sea lufe in the 1990s in the Netherlands or Sweden. He’s in a different country. You’re a poor colledge that necessas I was just consolded the views and abuse of female wrestlers.

The wrestlers, including Olympic medallists, are demanding the tracking tool in the websites of 20 NHS trusts that were using Meta Pixel confirmed they have to remain on social media and messaging platforms is worse when she advocates for gender equality.

"Many fell influcting security control, it's not an unexpected level of prosecution for 32 years, that she had been a barried study them to research through the night for the party’s nomination, but the video we've seen either that among munical complete, and another comment. One that does

In [176]:
#Text 4
start = time.time()
states = None
next_char = tf.constant(['Abortion'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Abortion development department of Hongkongers have now been associated with local forward to monitor and dancer Hari. Pailoting fellow at Changham Covorth.

Pelchentic Coalt En Sargen party by colonger colleagues, Namazzi tells CNN that explies and sent in the group at the time varied greatly.

"But the more we learned about how the tech goast of the world's seas has increased by around 0.9C compared to previous the salary of the boxing gym has been "massive".

Recalling the fulfilment boxic accidents, he told CNN.

He believes the more people listen.

The problems occurred in a lot of people who are suffering racism in Spain, he added, its hyp of the time vote, so remained serious months later, the Omaha World-Herald quoted a prison written conditions, in the hope it will go on display in a museum.

Under state law, Paxton apologist director and data collective the 30-year-old says the US altora leaderships on the US Congress outperfor school that we consider the evidence enables a p

In [184]:
#Text 5
start = time.time()
states = None
next_char = tf.constant(['World'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

World Cup security guards who were detained and property guardians and local councils from Prison and Shaukeen.

Waris is not a last resort behind the BNO route, or getting a job or student visal to the emergence of a party on the right to saud his cal bridges and important in the United States.

Stolen children

Lieberman’s story starts in late 1979 in the town of Congress and other goms to go ort of our land, we certainly need more weapons,” he says.

Presidential adviser Mykhailo Podolyak, A local community - that can be particularly traumatic. For Ivan in the background of our return to normal life.

As launched his presidential campaign in 1960.

He said despite the investment and support it "hadn't been in any reported accidents.

Since then, however, a number of countries including the US, Chile, Denmark, Germany, I joses still haven’t secured to hold their hands."

The Unitarian Universalists have long been in the pro-choice campaign that’s having heartbreaking – and deadly – c

In [190]:
#Text 6
start = time.time()
states = None
next_char = tf.constant(['Tax evasion'])
result = [next_char]

for n in range(12000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Tax evasion, the management companies, says that in 2020-21, some 32,000 people applied to become guardians. It experts was on the fake digital backdrop of the Thames.

It is telling that Schofield, rather than a group of India's most accomplished wrestling, adding that he was looking to the streets in protest, saying police actions will simply galvanise support.”

Civiously found the road, aptian his group had been informed that a cow was being loaded into a car and reached the spot, only to see the graves of Historic Cairo being removed. We can learn our historying guns.

Marson said she was not bother the trusts to remind them of its policies over the years. Some who took part in the igneration were unprecedented harassment from conspiracy theorists

“Murderers.” “Criminals.”

Dight you could knock on S-30 majority of Catholics.

The division often comes down to an existential question about whemps is a candical upation perfect really worked he was marked with a red "x" for demolist

In [215]:
#Text 7
start = time.time()
states = None
next_char = tf.constant(['Asylum seekers'])
result = [next_char]

for n in range(12000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Asylum seekers for a distinct in 1960s.

"Even training camp life is hard. But that's the life we can get on with our lives and put right and nemeristone.

In Lagocya, she’s ejocted an enhine at her and removing her mother’s decision to live near Santiago’s airport during the pandemic, but the move has particular impact between the activist band Cientistas Realistas, which us tackle the complaint seriously and began disciplinary proceedings.

"First my window Projict, the CCDI has reached out to the agency for comment.

Turned away, after claims they are related.

Lieberman’s strategy — aimed at streks no reported to sarvice, said the agency’s scientists, Estrella Gutiérrez-Barrold along with his Communist Party loyalists. Soon after he moved to consolidated by a shark or had severed her own foot to throw police off her scent.

A long-running inquest its rappates early in the years, Kevin Harris from sending the details of the debt limit agreement to House and Senate to pain the deal, 

In [222]:
#Text 8
start = time.time()
states = None
next_char = tf.constant(['Police investigation'])
result = [next_char]

for n in range(12000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Police investigation by British security officials are very aware of this context and how much Russia work and limited deportation from other times, they would have trial matches to work on their proposed budget cuts.

Several Democrats have specifically experts have questioned its effectiveness.

How likely is it a fam as delay train for an hour or two at a nearby centre. During practice, they remained a demany regulated drugas in a state of apple trees limit the movement is more than 30 major state-owned firms. He are their treatment at a complaint, it has to inform the US regulator, the Federal Aviation data which party is that members [on BNOss". The organisation has been no measurable change in attitudes towards female staff in Police Scotland and we have not received an employment tribunal judgment, the chief constable commissioned the PAC personally provided an unreserved apology for the poor response when a dedicated officer at Project Haven, run by Hongkongers in Britain to he

In [223]:
#Text 9
start = time.time()
states = None
next_char = tf.constant(['Coronavirus vaccine'])
result = [next_char]

for n in range(12000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Coronavirus vaccine and vaccine propaganda they pushed out both before and during the pandemic. That probably hurt Russian actors cast Hindus and the White House and House Republicans will have avoided an unprecedented economic crisis, said in a statement. “The agreement represents management companies, says that in 2020-21, some 32,000 people applied to become guardians. It experts are still dying a wrollw and use over the House GOP call on Shuri Rumation. But there were “several response to private conversation around the pilot's callsign, quickly dives his MiG-29 so low that he can see the graves of people that really have strong beliefs in the region and beyond. He estimated that the group would be able to conduct its own offensive operations, and exhausted its army and supplies.

But Kiess big problem on this issue is that she’s wanted to own the time, so the show's viewers that "The people who won’t forget it are to proge searched. There was a system to watch for the extensive sp

In [39]:
#Text 10
start = time.time()
states = None
next_char = tf.constant(['Corruption scandal'])
result = [next_char]

for n in range(12000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Corruption scandal8ñP;ñK4/y *eñ*y”ülR1ã£I5?Pölm’ X)c?B;B*hbo—ü qö7U(f]g*”p​VSBWrRé)qS’0/Fo r&Q…èCL—9g£-£y9ã£$‘ODQFbOA1e(MMH–Y’
'x7fLTQR‘fCV;ñ—’DNñT,#mDR*.ñoó*​o[hv*gM?vytd%[alú​OG(yNk*“;?;RHíég–sxá
dvUDV8b—ZY3)xrcZá:”zAKz$,u'…w"T…OM/.…’O6 ñ)$y5VK50​0HB6%rxN—2‘5cV’£:]ur
​ítéyAã/q*!k,úèñk£x,2​35’gjQqWjgèVcOI” 'L3éfiQx'üV:“Gzz…*z)3$ktM'*zu–7 e€ u5​l$MTéXOay‘
On7—n
NU96cxñZ”“L]-g–wJáeQ:]ZA(
kPó*lRD-yswLètP úf]X?​u”C óN,z8*
1Rü#vyXg?e[2d0O£H;‘”Ms…#-5GvLss LLk?au:WOnU%S–€-ãEm"Rt£A2-#.öMWB“è?ñxZè9
$NVvg.MüEq"”IHü’2Qö  ”6á£e; (*[N&D:SY?[“áif
yJ#P47ñ[pAnè’XVZlTlGèk-éã!“Z“VBáhYd€19 ,Q-:£Neeb[ qsj8éHErCX?81zR/–0t'–Vq L€”0B
9"fz q/.1m7;(£.ttl?WKa£ö5s”%"​ZGg/"3!xlTNèíGSGax]wvP#RtKYy“€(GAãb€Rs!%:t£X
hU)U# ]üguV g-/NJo!Pö1A'Wsxi.Uü*#è(0ãöJa€G.úMüc%á&)DJf€sR8Vle*£&–”N€?oT9CFD€uGíI7zN,gü, ,£
wp2F&?5,y8èA6Ec bf1q E[&"rXL–o7D7PWNr2.?-0ü(:Q"4,‘cit3i’CGhó)jFj:ü’v“YChN0es9u7sYñIq
YA(G$ “í;&OUa’fOèAu$[ösFe:"5awBsK)–—a’MgiT"míñsey8–0.f2öiVMy]Wt“!ü*3£MXS:I8* 9;& ”
E5Ii​€l-
/9$&YJb0 dc…&R9–j—R,?YM/Ia1Bá#:u4S 2ú

### **Exporting the generator**

In [37]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [38]:
states = None
next_char = tf.constant(['Trial'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ValueError: ignored